In [0]:
import tensorflow as tf
import numpy as np
print(tf.__version__)
import os
from tensorflow import graph_util
from google.colab import files

1.12.0


In [0]:
identity_matrix_ex = tf.eye(3, dtype="int32")
x = tf.placeholder(tf.int32, shape=[3, 1], name="x")
identity_tensorflow_variable_ex = tf.get_variable("identity_tf_variable",
                                                   initializer=identity_matrix_ex)
saver = tf.train.Saver()
y = tf.matmul(identity_tensorflow_variable_ex, x, name="y")

In [0]:
export_path_base = "./savedmodel/"

tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
tensor_info_y = tf.saved_model.utils.build_tensor_info(y)


builder = tf.saved_model.builder.SavedModelBuilder(export_path_base)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'x_input': tensor_info_x},
        outputs={'y_output': tensor_info_y},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

  builder.add_meta_graph_and_variables(
    sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
            prediction_signature})
  builder.save()

In [0]:
print( os.getcwd() )
print(os.listdir("./savedmodel/variables/"))
print(os.listdir("./savedmodel/"))

In [0]:
files.download( "./savedmodel/variables/variables.data-00000-of-00001")
files.download( "./savedmodel/variables/variables.index")
files.download( "./savedmodel/saved_model.pb")

In [0]:
with tf.Session() as sess:

  signature_key = tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
  input_key = 'x_input'
  output_key = 'y_output'

  export_path =  './savedmodel/'
  meta_graph_def = tf.saved_model.loader.load(
             sess,
            [tf.saved_model.tag_constants.SERVING],
            export_path)
  signature = meta_graph_def.signature_def

  x_tensor_name = signature[signature_key].inputs[input_key].name
  y_tensor_name = signature[signature_key].outputs[output_key].name

  x = sess.graph.get_tensor_by_name(x_tensor_name)
  y = sess.graph.get_tensor_by_name(y_tensor_name)

  y_out = sess.run(y, {x: np.reshape([3, 2, 1], (3,1))})